Recitation 9: Buffer Overflows (Ghidra)
==================================

Introduction
------------
In this lab we will be learning how to use Ghidra to conduct buffer overflow attacks.

Buffer overflow is when the attacker overflows the data by filling the buffer with more data than the capacity of the buffer itself. An attacker causes a buffer overflow by writing more than its length into the program's buffer, which corrupts the program's stack and causes the program to execute other instructions instead for the purpose of an attack. Functions that write data with no limit, and are therefore vulnerable to this attack, include sprintf, gets, scanf, and strcpy, among others.

As one of the most common memory vulnerabilities, it is the most powerful and classic category of vulnerability exploits used by attackers to compromise systems. Using buffer overflow attacks can lead to process failure, system downtime, reboots, and so on. More seriously, it can be exploited to execute unauthorized commands or even gain system privileges and thus perform various illegal operations.


Obtain a vulnerable executable
---------------
First, we need an executable file to attack. On Canvas, there is a binary called "recitation_9" which should run just fine on coding.csel.io. If you would prefer to compile the binary from source, it is also available with the file "recitation_9.c" and you can compile it with the line `gcc -g -m32 -static -Wno-implicit-function-declaration -U_FORTIFY_SOURCE recitation_9.c -fno-stack-protector -o recitation_9 -fcf-protection=none` (these flags are needed to disable some protections against this attack), however running it locally may result in additional complications and we suggest you stick to the cloud environment.

Inspect the file
----------------
Open the file in Ghidra and inspect the source (you can also read the source directly from the link above). The program asks for a password, however there is no valid way to log into an account! Our goal will be to call the "login" function.

Conduct the attack 
------------------
- Open the "recitation_9" function in Ghidra. In the central assembly window, see the comment Ghidra has added above the function definition. It shows the position of all local variables and function arguments, relative to the base pointer (values are in hex). This is right after the function is called: it assumes the base pointer is at the bottom of the stack and points to the return address which was pushed when the function was called.

- Draw out a diagram of the stack, and make sure you know the relative locations of the buffer `buf` and the return address.

- Find the address of the first instruction of the `login` function, which appears to the right of that instruction. This is the address we want to overwrite the return value with.  

- We can now perform the buffer overflow. Create an input that fills up the buffer and overwrites the return address with the one you just found. One easy way to do this is by using Python: you can create strings of a certain number of characters to fill the buffer with the syntax `'A' * 10`, and print hex bytes with the escape sequence `'\x12'`. A full attack might look something like `print('A' * 12 + '\x12\x34\x56\x78')`. If you are successful, it should print "Welcome, user!".
  - You will need to reverse the order of the bytes in the return address, as this is a little-endian program.
  - If you are having issues, you can debug the attack in GDB.

- We would like to make one last change to the attack- the `login` function takes an argument, `int is_admin`. If `is_admin == 1`, it logs the user in as an admin. Looking at Ghidra's comment above the function assembly, we can see it reads the argument off the stack, 4 bytes above the base pointer (directly above the return address). Modify your attack to add this argument. If you are successful, it should print "Welcome, admin!".
  - You will need to reverse the byte order here as well, so you will write it as `\x01\x00\x00\x00`.
  - You overwrite the base pointer in the initial attack, but it gets reset at the start of the `login` function.
  - It will be helpful to draw out the state of the stack after your attack, to know where the base pointer is.